In [77]:
import os
from os.path import join as pjoin
from tqdm import tqdm
import numpy as np

import sys
PROJECT_DIR = pjoin('..', '..') # Few-Shot_MDM
sys.path.append(PROJECT_DIR) # /root/of/Few-Shot_MDM

In [78]:

DATASET = 'NTU60'
SRC_DIR = pjoin(PROJECT_DIR, 'data' , DATASET, 'annotations')
TGT_ANI_DIR = pjoin('.', 'media')

NUM_ANIMATIONS = 3 # number of animations fromSRC_DIR to render into TGT_ANI_DIR

#SRC_DIR = pjoin(PROJECT_DIR, 'external' , 'motion-diffusion-model', 'dataset', 'HumanML3D', 'new_joint_vecs')

In [79]:
import pickle
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import mpl_toolkits.mplot3d.axes3d as p3

from utils.constants.skel import SKEL_INFO
from utils.constants.data import DATA_FILENAME
from scripts.skel_adaptation import backward_map

SKL = SKEL_INFO['HML3D']
FOOT_IDXS = SKL.fid_l + SKL.fid_r 

def plot_3d_motion(save_path, kinematic_tree, joints, title, figsize=(10, 10), fps=120, radius=4, foot_contacts=None):
#     matplotlib.use('Agg')

    title_sp = title.split(' ')
    if len(title_sp) > 10:
        title = '\n'.join([' '.join(title_sp[:10]), ' '.join(title_sp[10:])])
    def init():
        ax.set_xlim3d([-radius / 2, radius / 2])
        ax.set_ylim3d([0, radius])
        ax.set_zlim3d([0, radius])
        # print(title)
        fig.suptitle(title, fontsize=20)
        ax.grid(b=False)

    def plot_xzPlane(minx, maxx, miny, minz, maxz):
        ## Plot a plane XZ
        verts = [
            [minx, miny, minz],
            [minx, miny, maxz],
            [maxx, miny, maxz],
            [maxx, miny, minz]
        ]
        xz_plane = Poly3DCollection([verts])
        xz_plane.set_facecolor((0.5, 0.5, 0.5, 0.5))
        ax.add_collection3d(xz_plane)

    #         return ax

    # (seq_len, joints_num, 3)
    data = joints.copy().reshape(len(joints), -1, 3)
    fig = plt.figure(figsize=figsize)
    ax = p3.Axes3D(fig)
    init()
    MINS = data.min(axis=0).min(axis=0)
    MAXS = data.max(axis=0).max(axis=0)
    colors = ['red', 'blue', 'black', 'red', 'blue', 'darkred', 'darkblue'] 

    frame_number = data.shape[0]
    #     print(data.shape)

    height_offset = MINS[1]
    data[:, :, 1] -= height_offset
    trajec = data[:, 0, [0, 2]]
    
    data[..., 0] -= data[:, 0:1, 0]
    data[..., 2] -= data[:, 0:1, 2]

    #     print(trajec.shape)

    def update(index):
        #         print(index)
        ax.lines = []
        ax.collections = []
        ax.view_init(elev=120, azim=-90)
        ax.dist = 7.5
        #         ax =
        plot_xzPlane(MINS[0]-trajec[index, 0], MAXS[0]-trajec[index, 0], 0, MINS[2]-trajec[index, 1], MAXS[2]-trajec[index, 1])
        ax.scatter(data[index, :22, 0], data[index, :22, 1], data[index, :22, 2], color='black', s=3)
        
        if foot_contacts is not None:
            contact_flags = foot_contacts[index]            # (4,)
            for idx, flag in zip(FOOT_IDXS, contact_flags):
                if flag:
                    ax.scatter(data[index, idx, 0],
                               data[index, idx, 1],
                               data[index, idx, 2],
                               color='lime', s=40)

        if index > 1:
            ax.plot3D(trajec[:index, 0]-trajec[index, 0], np.zeros_like(trajec[:index, 0]), trajec[:index, 1]-trajec[index, 1], linewidth=1.0,
                      color='blue')
        #             ax = plot_xzPlane(ax, MINS[0], MAXS[0], 0, MINS[2], MAXS[2])
        
        
        for i, (chain, color) in enumerate(zip(kinematic_tree, colors)):
#             print(color)
            if i < 5:
                linewidth = 4.0
            else:
                linewidth = 2.0
            ax.plot3D(data[index, chain, 0], data[index, chain, 1], data[index, chain, 2], linewidth=linewidth, color=color)
        #         print(trajec[:index, 0].shape)

        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_zticklabels([])

    ani = FuncAnimation(fig, update, frames=frame_number, interval=1000/fps, repeat=False)

    ani.save(save_path, fps=fps)
    plt.close()


In [80]:
import numpy as np
import torch
from utils.humanml3d.skeleton import Skeleton
from utils.humanml3d.paramUtil import t2m_raw_offsets, t2m_kinematic_chain
from utils.pos2red_feat import recover_from_rot, recover_from_ric   # unchanged

def redundant_to_xyz(
        feat: np.ndarray,
        use_rotation: bool = False,
        device: str = "cpu",
):
    """
    Decode a (T-1, 263) redundant feature matrix into global XYZ joints
    and (optionally) foot-contact flags.

    Parameters
    ----------
    feat : np.ndarray
        Shape (T-1, 263) - output of `compute_redundant_motion_features`.
    use_rotation : bool, default False
        • True  - exact reconstruction via `recover_from_rot`
        • False - slightly faster, uses only local-pos branch (`recover_from_ric`)
    device : {"cpu", "cuda"}, default "cpu"
        Device on which to run the FK pass.

    Returns
    -------
    joints : np.ndarray, shape (T, 22, 3)
    contacts : np.ndarray, shape (T, 4), optional
        Returned only if `return_contacts=True`.
    """
    assert feat.ndim == 2 and feat.shape[1] == 263, "input must be (T-1, 263)"

    # ---------------- wrap as batch = 1 ----------------
    feat_t = torch.from_numpy(feat).float().unsqueeze(0).to(device)   # (1,T-1,263)

    # ---------------- decode joint positions -----------
    if use_rotation:
        # ---------------- build canonical skeleton ----------
        raw_offsets = torch.from_numpy(t2m_raw_offsets).to(device)        # (22,3)
        skel = Skeleton(raw_offsets, t2m_kinematic_chain, device)
        skel.set_offset(raw_offsets)   # give FK its bone lengths
        pos = recover_from_rot(feat_t, joints_num=22, skeleton=skel)  # (1,T-1,22,3)
    else:
        pos = recover_from_ric(feat_t, joints_num=22)                 # (1,T-1,22,3)

    joints = pos.squeeze(0).cpu().numpy()                             # (T-1,22,3)

    # prepend first frame (zero-velocity assumption)
    joints = np.concatenate([joints[[0]], joints], axis=0)            # (T,22,3)

    # ---------------- extract foot-contact flags -------
    #   The last 4 channels of the redundant vector are
    #   [L-heel, L-toe, R-heel, R-toe] ∈ {0,1}
    contacts = (feat[:, -4:] > 0.5).astype(bool)        # (T-1,4)  → bool
    contacts = np.concatenate([contacts[[0]], contacts], axis=0)      # (T,4)

    return joints, contacts


In [81]:
forw_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]
back_kinematic_chain = [[0, 14, 15, 16, 17], [0, 10, 11, 12, 13], [0, 1, 18, 2, 3], [18, 7, 8, 9], [18, 4, 5, 6]] 
orig_kinematic_chain = [[0, 12, 13, 14, 15], [0, 16, 17, 18, 19], [0, 1, 20, 2, 3], [20, 4, 5, 6], [20, 8, 9, 10], [21, 7, 6, 22], [23, 11, 10, 24]]

---

In [83]:
USE_ROTATION = False
OVERWRITE = True # whether to overwrite existing files

orig_filepath = pjoin(PROJECT_DIR, 'data', DATASET, DATA_FILENAME[DATASET])
print(f"Loading {DATASET} original annotations from {orig_filepath} ...")
with open(orig_filepath, 'rb') as file:
    orig_annotations = pickle.load(file)['annotations']

npy_files = sorted(os.listdir(SRC_DIR))[:NUM_ANIMATIONS]
for npy_file in tqdm(npy_files):
    joints_forw = np.load(pjoin(SRC_DIR, npy_file))
    name = npy_file.split('.')[0]

    joints_forw, contacts = redundant_to_xyz(joints_forw, use_rotation=USE_ROTATION)
    joints_back = backward_map(joints_forw)
    joints_orig = next( # NOTE: only first skeleton is plotted
        (data['keypoint'][0] for data in orig_annotations if data['frame_dir'] == name),
        None
    )
    assert joints_orig is not None, f"Original joints for {name} not found in annotations."

    forw_save_path = pjoin(TGT_ANI_DIR, DATASET, name, 'forw.mp4')
    backw_save_path = pjoin(TGT_ANI_DIR, DATASET, name, 'back.mp4')
    orig_save_path = pjoin(TGT_ANI_DIR, DATASET, name, 'orig.mp4')
    os.makedirs(os.path.dirname(forw_save_path), exist_ok=True)
    os.makedirs(os.path.dirname(backw_save_path), exist_ok=True)
    os.makedirs(os.path.dirname(orig_save_path), exist_ok=True)

    if not OVERWRITE and (os.path.exists(forw_save_path) or os.path.exists(backw_save_path) or os.path.exists(orig_save_path)):
        continue

    radius = 2.0 if not USE_ROTATION else 12.0
    # You may set the title on your own.
    plot_3d_motion(forw_save_path, forw_kinematic_chain, joints_forw, title="SMPL (Forw) - 20fps", fps=20, radius=radius, foot_contacts=contacts)
    plot_3d_motion(backw_save_path, back_kinematic_chain, joints_back, title="Kinect-reduced (Backw) - 20fps", fps=20, radius=radius, foot_contacts=None)
    plot_3d_motion(orig_save_path, orig_kinematic_chain, joints_orig, title="Kinect (Orig) - 30fps", fps=30, radius=radius, foot_contacts=None)

Loading NTU60 original annotations from ../../data/NTU60/ntu60_3danno.pkl ...


100%|██████████| 3/3 [00:54<00:00, 18.12s/it]


<br><br>

---
---
---